In [12]:
import numpy as np
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister
from qiskit import BasicAer
from qiskit import execute
from itertools import product
import math
%matplotlib inline

In [13]:
# Number of qubit to be considered
n_qubit = 5

# Draw a random binary vector in length of n_qubit to define 
# the true product state that needs to be reconstructed
true_draw = np.random.randint(2,size=n_qubit)

In [14]:
def prepare_circuits(n_qubit,true_draw):
    """
    This function creates a list of quantum circuits, one circuit for one grid point.
    Each quantum circuit consists of:
    
    1, true_qr, represents the true multi-qubit state to be reconstructed. In the circuit, true_qr passes through 
    the Pauli-X gate according to the input true_draw argument.
    
    2, grid_qr, represents the multi-qubit state corresponding to a grid point. Again, grid_qr passes through 
    the Pauli-X gate according to grid configuration.
    
    3, anc_qr and cr, represents ancilla qubit and classical bit for measurement respectively
    
    Note: I wonder if there is any way to define a varitional circuit to configure grid_qr so that we do not 
    need to define one quantum circuit for each grid point.
    
    return: a list of ntuple with 2 elements: grid configuration and the corresponding quantum circuit.
    """
    circuit_list = []
    grids = list(product(range(2), repeat=n_qubit))
    for grid in grids:
        true_qr = QuantumRegister(n_qubit,'true_qr')
        grid_qr = QuantumRegister(n_qubit,'grid_qr')
        anc_qr = QuantumRegister(1,'anc_qr')
        cr = ClassicalRegister(1,'c')
        
        qc = QuantumCircuit(true_qr,grid_qr,anc_qr,cr)
        
        for i in range(n_qubit):
            if true_draw[i]: qc.x(i)
            if grid[i]: qc.x(n_qubit+i)
            qc.h(anc_qr)
            qc.cswap(anc_qr,true_qr[i],grid_qr[i])
            qc.h(anc_qr)
            qc.measure(anc_qr,cr)
                
        circuit_list.append((grid,qc))
        
    return circuit_list

In [15]:
# Make all the circuits needed
circuits = prepare_circuits(n_qubit,true_draw)

In [16]:
# Draw one circuit to inspect
circuits[5][1].draw()

»
true_qr_0: ──────X─────────────────────────────────────────────────────────»
                 │                                                         »
true_qr_1: ──────┼───────────────X─────────────────────────────────────────»
           ┌───┐ │               │                                         »
true_qr_2: ┤ X ├─┼───────────────┼───────────────X─────────────────────────»
           └───┘ │               │               │                         »
true_qr_3: ──────┼───────────────┼───────────────┼───────────────X─────────»
           ┌───┐ │               │               │               │         »
true_qr_4: ┤ X ├─┼───────────────┼───────────────┼───────────────┼─────────»
           └───┘ │               │               │               │         »
grid_qr_0: ──────X───────────────┼───────────────┼───────────────┼─────────»
                 │               │               │               │         »
grid_qr_1: ──────┼───────────────X───────────────┼───────────────┼─────────»
           ┌───┐ │               │               │               │         »
grid_qr_2: ┤ X ├─┼───────────────┼───────────────X───────────────┼─────────»
           └───┘ │               │               │               │         »
grid_qr_3: ──────┼───────────────┼───────────────┼───────────────X─────────»
           ┌───┐ │               │               │               │         »
grid_qr_4: ┤ X ├─┼───────────────┼───────────────┼───────────────┼─────────»
           ├───┤ │ ┌───┐┌─┐┌───┐ │ ┌───┐┌─┐┌───┐ │ ┌───┐┌─┐┌───┐ │ ┌───┐┌─┐»
 anc_qr_0: ┤ H ├─■─┤ H ├┤M├┤ H ├─■─┤ H ├┤M├┤ H ├─■─┤ H ├┤M├┤ H ├─■─┤ H ├┤M├»
           └───┘   └───┘└╥┘└───┘   └───┘└╥┘└───┘   └───┘└╥┘└───┘   └───┘└╥┘»
      c: 1/══════════════╩═══════════════╩═══════════════╩═══════════════╩═»
                         0               0               0               0 »
«                           
«true_qr_0: ────────────────
«                           
«true_qr_1: ────────────────
«                           
«true_qr_2: ────────────────
«                           
«true_qr_3: ────────────────
«                           
«true_qr_4: ──────X─────────
«                 │         
«grid_qr_0: ──────┼─────────
«                 │         
«grid_qr_1: ──────┼─────────
«                 │         
«grid_qr_2: ──────┼─────────
«                 │         
«grid_qr_3: ──────┼─────────
«                 │         
«grid_qr_4: ──────X─────────
«           ┌───┐ │ ┌───┐┌─┐
« anc_qr_0: ┤ H ├─■─┤ H ├┤M├
«           └───┘   └───┘└╥┘
«      c: 1/══════════════╩═
«                         0

In [20]:
# Run simulation on all circults and put the results into out_list

shots = 10
out_list = []
for grid,qc in circuits:
    
    # Run the circuit n times
    backend = BasicAer.get_backend('qasm_simulator')
    results = execute(qc, backend=backend, shots=shots).result()
    answer = results.get_counts()
    
    # Calculate swap result
    sumM = float(answer['1']) if '1' in answer else 0.
    s = 1.-2./shots*sumM
        
    # Save result of this grid point to the list
    out_list.append([grid,s])
    
# Sort out_list with s
out_list.sort(key=lambda x: x[1], reverse=True)
# Print out the grid with max s
print("Estimated Configuration: ",out_list[0][0])
print("True Configuration: ",true_draw)

Estimated Configuration:  (0, 0, 1, 0, 1)
True Configuration:  [0 0 1 0 1]
